Copyright © 2022, SAS Institute Inc., Cary, NC, USA.  All Rights Reserved. 
SPDX-License-Identifier: Apache-2.0

# Configure a Google Cloud Platform Container Publishing Destination

_**Important:** This example should be used for SAS Viya 2021.2.3 and later stable releases._

In order to create a Google Cloud Platform destination, you must complete the following steps:

1. Establish a connection and define URLs.
2. Get an authorization token.
3. Find a valid domain with Google Cloud credentials.
4. Create credentials for a specific user or group and define a credential domain.
5. Submit an API post to create a Google Cloud Platform destination.

_Note: An example of deleting a destination is also included._

## Establish a Connection and Define URLs

In [ ]:
import sys
sys.path.append('..')
import mmAuthorization

import requests
import json, os, pprint
import base64

In [ ]:
host_name= "localhost"
port = "8080"

host_url="http://" + host_name + ":" + port
destination_url = host_url + "/modelPublish/destinations/"
modelrepo_url = host_url + "/modelRepository/models/"
publishmodel_url = host_url + "/modelPublish/models"
domains_url = host_url + "/credentials/domains"
print(host_url)

## Get an Authorization Token and Define Headers

In [ ]:
mm_auth = mmAuthorization.mmAuthorization("myAuth")

admin_userId = '<SAS_user_admin_ID>'
user_passwd =  '<SAS_user_password>'

sas_userId = '<SAS_user_ID>'

admin_auth_token = mm_auth.get_auth_token(host_url, admin_userId, user_passwd)

credential_admin_headers = {
    mmAuthorization.AUTHORIZATION_HEADER: mmAuthorization.AUTHORIZATION_TOKEN + admin_auth_token
}

credential_domain_headers = {
    "If-Match":"false",
    "Content-Type":"application/json",
    mmAuthorization.AUTHORIZATION_HEADER: mmAuthorization.AUTHORIZATION_TOKEN + admin_auth_token
}

credential_user_headers = {
    "If-Match":"false",
    "Content-Type":"application/json",
    mmAuthorization.AUTHORIZATION_HEADER: mmAuthorization.AUTHORIZATION_TOKEN + admin_auth_token
}

destination_azure_headers = {
    "If-Match":"false",
    "Content-Type":"application/vnd.sas.models.publishing.destination.azure+json",
    mmAuthorization.AUTHORIZATION_HEADER: mmAuthorization.AUTHORIZATION_TOKEN + admin_auth_token
}

print(admin_auth_token)

## Create a Credential Domain

In [ ]:
domain_name = '<Domain_Name>'
description = '<Domain_Description>'

my_domain_url = domains_url + "/" + domain_name
domain_attrs = {
    "id": domain_name,
    "type": "base64",
    "description": description
}

domain = requests.put(my_domain_url, 
                       data=json.dumps(domain_attrs), headers=credential_domain_headers)

print(domain)
pprint.pprint(domain.json())

## Create User Credentials

In [ ]:
domain_name = '<Domain_Name>'
my_domain_url = domains_url + "/" + domain_name

user_credential_name = sas_userId
my_credential_url = my_domain_url + "/users/" + user_credential_name

clientID = '<GCP_Service_Account_ID>'
clientSecret = '<GCP_Service_Account_JSON_Key>'

encoded_clientID = str(base64.b64encode(clientID.encode("utf-8")), "utf-8")
encoded_clientSecret = str(base64.b64encode(clientSecret.encode("utf-8")), "utf-8")

credential_attrs = {
    "domainId":domain_name,
    "identityType":"user",
    "identityId":user_credential_name,
    "domainType":"base64",
    "properties":{"serviceAccount":encoded_clientID},
    "secrets":{"credentialJson":encoded_clientSecret}
}

credential = requests.put(my_credential_url, 
                       data=json.dumps(credential_attrs), headers=credential_user_headers)

print(credential)
pprint.pprint(credential.json())

## Create a Google Cloud Platform Destination

In [ ]:
dest_name = '<Destination_Name>'
domainName = '<Domain_Name>'
baseRepoUrl = '<GCP_Container_Registry_Base_URL>' # Typically: gcr.io/<subsciption-ID> (i.e. gcr.io/solorgasub7)
k8sClusterName = '<GCP_Kubernetes_Service_Cluster_Name>' # The name of the cluster used for validation.
region= '<region>' # The "Control plane zone" associated with the cluster.

destination_attrs = {
    'name': dest_name,
    'destinationType': 'gcp',
    'properties': [{'name': "credDomainId", "value": domainName},
                   {"name": "baseRepoUrl", "value": baseRepoUrl},
                   {'name': "kubernetesCluster", "value": k8sClusterName},
                   {'name': "clusterLocation", "value": region}]}

destination = requests.post(destination_url, 
                       data=json.dumps(destination_attrs), headers=destination_azure_headers)

print(destination)
pprint.pprint(destination.json())

## Delete a Destination

In [ ]:
deletedURL = destination_url + dest_name

destination = requests.delete(deletedURL, headers=credential_admin_headers)

print(deletedURL)
print(destination)
pprint.pprint(destination.json())